LeNet

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l


# 重塑张量函数
class Reshape(torch.nn.Module):
    def forward(self,x):
        return x.view(-1,1,28,28)  # -1 在 view 方法中的意义是依据其他维度的值自动推算出该维度的大小


# LeNet
net = torch.nn.Sequential(
    Reshape(), nn.Conv2d(1,6,kernel_size=5,padding=2),nn.Sigmoid(),  # 第一个卷积层
    nn.AvgPool2d(2,stride=2),  # 第一个平均池化层
    nn.Conv2d(6,16,kernel_size=5),nn.Sigmoid(),  # 第二个卷积层
    nn.AvgPool2d(kernel_size=2,stride=2),nn.Flatten(),  # 第二个平均池化层
    nn.Linear(16*5*5, 120),nn.Sigmoid(),  # 输入层
    nn.Linear(120, 84),nn.Sigmoid(),  # 隐藏层
    nn.Linear(84, 10)  # 输出层
)

LeNet训练Fashion-MNIST数据集

In [ ]:
import torch
from torch import nn
from d2l import torch as d2l
import os

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'


class Reshape(torch.nn.Module):
    def forward(self,x):
        return x.view(-1,1,28,28)


# LeNet
net = torch.nn.Sequential(
    Reshape(), nn.Conv2d(1,6,kernel_size=5,padding=2),nn.Sigmoid(),  # 第一个卷积层
    nn.AvgPool2d(2,stride=2),  # 第一个平均池化层
    nn.Conv2d(6,16,kernel_size=5),nn.Sigmoid(),  # 第二个卷积层
    nn.AvgPool2d(kernel_size=2,stride=2),nn.Flatten(),  # 第二个平均池化层
    nn.Linear(16*5*5, 120),nn.Sigmoid(),  # 输入层
    nn.Linear(120, 84),nn.Sigmoid(),  # 隐藏层
    nn.Linear(84, 10)  # 输出层
)

# 加载数据集
help(d2l.load_data_fashion_mnist)

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)


# 使用GPU计算模型在数据集上的精度
def evaluate_accuracy_gpu(net, data_iter, device=None):
    if isinstance(net, torch.nn.Module):
        net.eval()  # net.eval()开启验证模式，不用计算梯度和更新梯度
        if not device:
            device = next(iter(net.parameters())).device  # 看net.parameters()中第一个元素的device为哪里
    metric = d2l.Accumulator(2)
    for X, y in data_iter:
        if isinstance(X,list):
            X = [x.to(device) for x in X]  # 如果X是个List，则把每个元素都移到device上
        else:
            X = X.to(device)  # 如果X是一个Tensor，则只用移动一次，直接把X移动到device上
        y = y.to(device)
        metric.add(d2l.accuracy(net(X),y),y.numel())  # y.numel()为 y元素个数
    return metric[0]/metric[1]


def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """Train a model with a GPU"""

    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)  # 根据输入、输出大小，使得随即初始化后，输入和输出的的方差是差不多的

    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)

    for epoch in range(num_epochs):
        metric = d2l.Accumulator(3)
        net.train()

        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]

            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))

    print(f'loss {train_l:.3f},train acc {train_acc:.3f},'f'test acc {test_acc:.3f}')

    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec'f'on{str(device)}')


help(nn.init.xavier_uniform_)

# 训练和评估LeNet-5模型
lr, num_epochs = 0.9, 10
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

d2l.plt.show()